# Text Mining 

Source: https://kgptalkie.com/3664-2/

In [2]:
import pandas as pd
import numpy as np

In [3]:
# pip install -U spacy
# pip install -U spacy-lookups-data
# pip install -U pip setuptools wheel

In [4]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

In [17]:
# Below 3 has to be installed from terminal (command prompt) not in Jupyter. 
# sm/md/lg refer to the sizes of the models (small, medium, large respectively).

# python -m spacy download en_core_web_sm  # 12mb
# python -m spacy download en_core_web_md  # 50mb
# python -m spacy download en_core_web_lg  # 800mb

# _lg gives more accuracy, due to huge size have not installed now. For normal practise small and medium package is
# enough. When going to prod we can make a minor change in the code to refer to large one as, 
# nlp = spacy.load("en_core_web_lg")

In [5]:
# nlp = spacy.load('en_core_web_sm')
nlp = spacy.load('en_core_web_md')

# General Feature Extraction

In [6]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding = 'latin1', header = None)

# Need to keep this file in Datasets folder and load it here. Its too huge. If its kept in this folder then it gets
# uploaded to github also, which is too big. 

In [7]:
df.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [8]:
df.shape

(1600000, 6)

In [9]:
df = df[[5, 0]]  # create new DF with only 5th and 1st column. 

In [10]:
df.head()

,5,0
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [11]:
df.columns = ['twitts', 'sentiment']
df.head()

,twitts,sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


In [12]:
df['sentiment'].value_counts()

4    800000
0    800000
Name: sentiment, dtype: int64

In [13]:
sent_map = {0: 'negative', 4: 'positive'}
sent_map

{0: 'negative', 4: 'positive'}

In [14]:
# Just keep a copy. 

original_df = df

In [15]:
original_df.head(5)

,twitts,sentiment
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0
4,"@nationwideclass no, it's not behaving at all....",0


## Word Counts

In [16]:
len('my whole body feels itchy and like its on fire')  # this is characters count including space. 

46

In [17]:
len('my whole body feels itchy and like its on fire'.split()) # exact words count excluding space

10

In [18]:
df['words_counts'] = df['twitts'].apply(lambda x: len(str(x).split()))

In [19]:
df.head()

,twitts,sentiment,words_counts
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19
1,is upset that he can't update his Facebook by ...,0,21
2,@Kenichan I dived many times for the ball. Man...,0,18
3,my whole body feels itchy and like its on fire,0,10
4,"@nationwideclass no, it's not behaving at all....",0,21


In [20]:
df.iloc[1]

# Be careful that the twitts shown below also not complete. 

twitts          is upset that he can't update his Facebook by ...
sentiment                                                       0
words_counts                                                   21
Name: 1, dtype: object

In [21]:
df.iloc[1].twitts

# we need to see complete text as below.

"is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"

In [22]:
len(df.iloc[1].twitts)

111

In [23]:
len(df.iloc[1].twitts.split())  # now we can see that the calculated word count is correct i.e 21

21

## Characters Count

In [24]:
df['char_counts'] = df['twitts'].apply(lambda x: len(x))

In [25]:
df.head()

,twitts,sentiment,words_counts,char_counts
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115
1,is upset that he can't update his Facebook by ...,0,21,111
2,@Kenichan I dived many times for the ball. Man...,0,18,89
3,my whole body feels itchy and like its on fire,0,10,47
4,"@nationwideclass no, it's not behaving at all....",0,21,111


## Average Word Length

In [26]:
def get_avg_word_len(x):
    words = x.split()
    word_len = 0
    for i in words:
        word_len = word_len + len(i)
    return word_len/len(words) # != len(x)/len(words)

In [27]:
df['avg_word_len'] = df['twitts'].apply(lambda x: get_avg_word_len(x))

df.head()

,twitts,sentiment,words_counts,char_counts,avg_word_len
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444
3,my whole body feels itchy and like its on fire,0,10,47,3.700000
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714


In [28]:
df.iloc[1].twitts

"is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"

In [29]:
words = df.iloc[1].twitts.split()

words, len(words)

(['is',
  'upset',
  'that',
  'he',
  "can't",
  'update',
  'his',
  'Facebook',
  'by',
  'texting',
  'it...',
  'and',
  'might',
  'cry',
  'as',
  'a',
  'result',
  'School',
  'today',
  'also.',
  'Blah!'],
 21)

In [30]:
word_len = 0

for i in words:
    word_len = word_len + len(i)
    

In [31]:
word_len   

# now we got word_len 90 instead of 111 because the statement is split into 21 words and spaces are 
# removed due to split function. This is the right way since it doesnt make sense to count the space while calculating
# the average. 

90

In [32]:
 90/21

4.285714285714286

## Stop Words Count

In [33]:
print(STOP_WORDS)

{'nor', 'everywhere', 'whither', 'six', 'becomes', 'others', 'forty', 'will', 'however', 'they', 'through', 'less', 'beyond', 'in', 'themselves', 'before', 'out', 'noone', 'anything', 'anyhow', '‘ll', 'herself', 'nobody', 'it', 'became', 'thereafter', 'has', 'still', 'along', 'nevertheless', 'of', '’ve', 'its', 'at', 'onto', 'had', 'up', 'sometimes', 'bottom', 'yet', '’d', 'whence', 'alone', 'four', 'last', 'were', 'nothing', 'since', 'we', 'about', '’re', "'d", 'twelve', 'hence', 'below', 'none', 'their', 'to', 'beside', 'few', 'this', 'please', 'whole', 'must', 'anywhere', 'did', 'his', 'every', 'namely', 'all', 'ourselves', 'too', 'how', 'not', '‘m', 'been', 'elsewhere', 'almost', 'rather', 'various', 'or', "'ll", 'either', 'should', 'you', 'except', 'whatever', 'these', 'an', 'take', 'am', 'against', 'done', 'even', 'never', 'seeming', 'being', 'could', 'also', 'anyway', 'call', 'part', 'something', 'nowhere', 'same', 'become', 'across', 'each', 'wherever', 'used', 'is', 'former', 

In [34]:
x = 'this is text data'

In [35]:
x.split()

['this', 'is', 'text', 'data']

In [36]:
len([t for t in x.split() if t in STOP_WORDS])

2

In [37]:
df['stop_words_len'] = df['twitts'].apply(lambda x: len([t for t in x.split() if t in STOP_WORDS]))

In [38]:
df.head()

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632,4
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714,9
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444,7
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10


## Count #HashTags and @Mentions

In [39]:
x = 'this #hashtag and this is @mention'

In [40]:
[t for t in x.split() if t.startswith('@')]

['@mention']

In [41]:
df['hashtags_count'] = df['twitts'].apply(lambda x: len([t for t in x.split() if t.startswith('#')]))

df['mentions_count'] = df['twitts'].apply(lambda x: len([t for t in x.split() if t.startswith('@')]))

In [42]:
df.head()

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,19,115,5.052632,4,0,1
1,is upset that he can't update his Facebook by ...,0,21,111,4.285714,9,0,0
2,@Kenichan I dived many times for the ball. Man...,0,18,89,3.944444,7,0,1
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0
4,"@nationwideclass no, it's not behaving at all....",0,21,111,4.285714,10,0,1


## Numeric Digits Count

In [43]:
df['numerics_count'] = df['twitts'].apply(lambda x: len([t for t in x.split() if t.isdigit()]))

In [44]:
df.sample(5)

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count
833494,working now. | home alone.,4,5,27,4.400000,0,0,0,0
111170,@drewryanscott ugh..Twit haha I hope you get ...,0,16,83,4.187500,8,0,1,0
1284588,@Heynewgirl Yep I have epilepsy and never hid ...,4,22,103,3.681818,10,0,1,0
504871,@gregcolombo im an home takin it easy. close t...,0,14,73,4.214286,5,0,1,0
1086855,ears dropping is fun,4,4,21,4.250000,1,0,0,0


In [45]:
df.iloc[354289].twitts

'came back from daiy plazza where president Kennedy Got Shot &amp; Killed in     1963 '

# UPPER case words count

In [46]:
# we are calculating the number of UPPERcase words in each sentences if length is more than 3.

In [47]:
df['upper_counts'] = df['twitts'].apply(lambda x: len([t for t in x.split() if t.isupper() and len(t)>3]))

In [48]:
df.sample(5)

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts
240010,last day of the weekend.,0,5,25,4.000000,3,0,0,0,0
676788,I haven't been able to sleep in on the weekend...,0,21,108,4.095238,12,0,0,0,0
407367,@symphnysldr agreed. I missed my party today tho,0,8,49,5.125000,1,0,1,0,0
1109580,At the nashville screenwriters conference... E...,4,14,89,5.357143,5,0,0,0,0
712473,Oh wow..This man in a car next to us..teeth li...,0,23,132,4.739130,5,0,0,0,1


In [49]:
df.iloc[67998].twitts

'@MatthewTLego OH MY GOSH I FORGOT TO WATCH THAT TONIGHT SKDJFHLSKDJFH WILL THEY PLAY IT AGAIN??? pwease say yes '

# Preprocessing and Cleaning

## Lower case conversion

In [50]:
x = 'THIS is Text'
x

'THIS is Text'

In [51]:
x.lower()

'this is text'

In [52]:
x = 45.0
x

45.0

In [54]:
x.lower()

AttributeError: 'float' object has no attribute 'lower'

In [55]:
# To overcome above issue we need to cast it to String.

str(x).lower()

'45.0'

In [56]:
df['twitts'] = df['twitts'].apply(lambda x: str(x).lower())

In [57]:
df.sample(5)

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts
460325,imma go to sleep really early tonight i'm soo...,0,13,82,5.307692,2,0,0,0,1
895895,@bradphoto yah know,4,3,20,5.666667,0,0,1,0,0
1069091,just got back from shipping off a bunch of the...,4,19,100,4.263158,8,0,0,0,0
859166,@triplebeamdream @mparachou thanks for includi...,4,9,74,7.222222,4,1,2,0,1
1353914,going to el mez with sadie and liz..then chap...,4,10,58,4.800000,3,0,0,0,0


## Contraction to Expansion

In [58]:
# In this section, we are converting all short words to their respective fullwords based on the words defined in the dictionary and using function 
# cont_to_exp().

In [59]:
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how does",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
" u ": " you ",
" ur ": " your ",
" n ": " and "}

In [60]:
x = "i don't know what you want, can't, he'll, i'd"

In [61]:
def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key, value)
        return x
    else:
        return x

In [62]:
cont_to_exp(x)

'i do not know what you want, cannot, he will, i would'

In [63]:
df['twitts'] = df['twitts'].apply(lambda x: cont_to_exp(x))

In [64]:
df.sample(5)

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts
278721,@arrens hehe. pretentious pricks need to be ta...,0,16,99,5.187500,5,0,1,0,0
707367,@50centfranks 103.5 i am not kidding thinking...,0,11,76,5.818182,3,0,1,0,1
1375208,@foliovision hey am loyal to wordpress,4,6,39,5.500000,2,0,1,0,0
252831,@surlyshirley sorry to disspoint you honestly...,0,21,136,5.476190,8,0,1,0,0
568389,"why is it, you always lose the receipt of the ...",0,22,106,3.818182,11,0,0,0,0


## Count and Remove Emails

In [65]:
import re

In [66]:
x = 'hi my email me at email@email.com another@email.com'

In [67]:
# Find the email id's using regular expression

re.findall(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', x)

['email@email.com', 'another@email.com']

In [68]:
df['emails'] = df['twitts'].apply(lambda x: re.findall(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', x))

In [69]:
df.sample(5)

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts,emails
305853,@bretfriedrich enzo??? if so i had no idea! yo...,0,13,68,4.230769,9,0,1,0,0,[]
1176570,@jordanknight you pack quick! must be going al...,4,23,138,5.000000,10,0,2,0,0,[]
176610,oh also... here's a clip of the new moon trail...,0,19,107,4.578947,6,0,0,0,0,[]
311425,@twin_1 aww man i forgot....why are you over...,0,14,80,4.642857,3,0,1,0,0,[]
508280,why cannot i sleep! ugh summer school tomorrow,0,8,46,4.750000,0,0,0,0,0,[]


In [70]:
df['emails_count'] = df['emails'].apply(lambda x: len(x))

In [71]:
df[df['emails_count']>0].head()

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts,emails,emails_count
4054,i want a new laptop. hp tx2000 is the bomb. :...,0,20,103,4.150000,6,0,0,0,1,[gabbehhramos@yahoo.com],1
7917,who stole elledell@gmail.com?,0,3,31,9.000000,1,0,0,0,0,[elledell@gmail.com],1
8496,@alexistehpom really? did you send out all th...,0,20,130,5.500000,11,0,1,0,0,[missataari@gmail.com],1
10290,@laureystack awh...that is kinda sad lol add ...,0,8,76,8.500000,0,0,1,0,0,[hello.kitty.65@hotmail.com],1
16413,"@jilliancyork got 2 bottom of it, human error...",0,21,137,5.428571,7,0,1,1,0,[press@linkedin.com],1


In [72]:
x

'hi my email me at email@email.com another@email.com'

In [73]:
# get the content by excluding email id.

re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', x)

'hi my email me at  '

In [74]:
df['twitts'] = df['twitts'].apply(lambda x: re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '', x))

In [75]:
df[df['emails_count']>0].head()

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts,emails,emails_count
4054,i want a new laptop. hp tx2000 is the bomb. :...,0,20,103,4.150000,6,0,0,0,1,[gabbehhramos@yahoo.com],1
7917,who stole ?,0,3,31,9.000000,1,0,0,0,0,[elledell@gmail.com],1
8496,@alexistehpom really? did you send out all th...,0,20,130,5.500000,11,0,1,0,0,[missataari@gmail.com],1
10290,@laureystack awh...that is kinda sad lol add ...,0,8,76,8.500000,0,0,1,0,0,[hello.kitty.65@hotmail.com],1
16413,"@jilliancyork got 2 bottom of it, human error...",0,21,137,5.428571,7,0,1,1,0,[press@linkedin.com],1


In [1]:
df.iloc[4054].twitts

NameError: name 'df' is not defined

## Count URLs and Remove it

In [77]:
x = 'hi, to watch more visit https://youtube.com/kgptalkie'

In [78]:
re.findall(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', x)

[('https', 'youtube.com', '/kgptalkie')]

In [79]:
re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x)

'hi, to watch more visit '

In [80]:
df['urls_flag'] = df['twitts'].apply(lambda x: len(re.findall(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', x)))

In [81]:
df[df['urls_flag']>0].head()

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts,emails,emails_count,urls_flag
0,"@switchfoot http://twitpic.com/2y1zl - awww, t...",0,19,115,5.052632,4,0,1,0,0,[],0,1
37,@missxu sorry! bed time came here (gmt+1) ht...,0,8,61,6.500000,1,0,1,0,1,[],0,1
50,broadband plan 'a massive broken promise' http...,0,15,129,7.600000,4,0,0,0,0,[],0,1
74,why won't you show my location?! http://twit...,0,7,59,7.285714,3,0,0,0,0,[],0,1
95,strider is a sick little puppy http://apps.fa...,0,7,85,11.142857,2,0,0,0,0,[],0,1


In [82]:
df.iloc[37].twitts

'@missxu sorry! bed time came here (gmt+1)   http://is.gd/fnge'

In [83]:
df['twitts'] = df['twitts'].apply(lambda x: re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', x))

In [84]:
df.iloc[37].twitts

'@missxu sorry! bed time came here (gmt+1)   '

## Remove RT (ReTweets)

In [85]:
x = 'rt @username: hello hi rtent'

In [86]:
re.sub(r'\brt\b','',x)

# only the word 'rt' gets removed. But in that place space is still remaining. 

' @username: hello hi rtent'

In [87]:
re.sub(r'\brt\b','',x).strip()

# now the space also got removed.

'@username: hello hi rtent'

In [88]:
df['twitts'] = df['twitts'].apply(lambda x: re.sub('rt', "", x).strip())

In [89]:
df.sample(5)

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts,emails,emails_count,urls_flag
993134,@alauderdale: hey sweethea. good morning.,4,5,44,7.800000,0,0,1,0,0,[],0,0
128541,trying to study for exams... going to fail,0,8,43,4.375000,3,0,0,0,0,[],0,0
884667,@mariaruizx she is most likely going to the tc...,4,9,55,5.111111,3,0,1,0,0,[],0,0
1567133,@nora1027 hey girl!!!!!!!! when you gonna slo...,4,16,95,4.812500,1,0,1,0,0,[],0,0
7679,@lenejohansen i had to reschedule bc a throat ...,0,13,79,5.076923,4,0,1,0,0,[],0,0


## Special Chars and Punctuation removal

In [90]:
df.head()

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts,emails,emails_count,urls_flag
0,"@switchfoot - awww, that is a bummer. you sh...",0,19,115,5.052632,4,0,1,0,0,[],0,1
1,is upset that he cannot update his facebook by...,0,21,111,4.285714,9,0,0,0,0,[],0,0
2,@kenichan i dived many times for the ball. man...,0,18,89,3.944444,7,0,1,0,0,[],0,0
3,my whole body feels itchy and like its on fire,0,10,47,3.700000,5,0,0,0,0,[],0,0
4,"@nationwideclass no, it is not behaving at all...",0,21,111,4.285714,10,0,1,0,0,[],0,0


In [91]:
df.iloc[2].twitts

'@kenichan i dived many times for the ball. managed to save 50%  the rest go out of bounds'

In [92]:
x= df.iloc[2].twitts

re.sub('[^A-Z a-z 0-9-]+', '', x)

# it removed % and full stop also. Now the statement doesn't give real meaning, not sure if its right to remove all
# special characters and full stops. Depends on use case also. But this is the approach to remove it. 

'kenichan i dived many times for the ball managed to save 50  the rest go out of bounds'

In [93]:
df['twitts'] = df['twitts'].apply(lambda x: re.sub('[^A-Z a-z 0-9-]+', '', x))

In [94]:
df.sample(5)

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts,emails,emails_count,urls_flag
844083,yenmenthol tell me about it have a goodnight,4,8,48,5.000000,3,0,1,0,0,[],0,0
279118,228 souls on board air france lost at sea,0,10,87,7.400000,2,0,0,1,0,[],0,1
337232,wants to know why if i have given up smoking b...,0,22,105,3.772727,11,0,0,0,0,[],0,0
661325,should have went back to jersey w mushi instea...,0,15,87,4.800000,5,0,0,0,0,[],0,0
826541,karencivil okay he just texted me saying that ...,4,26,126,3.846154,13,0,1,0,0,[],0,0


## Remove multiple spaces

In [95]:
x = 'thanks    for    watching and    please    like this video'

In [96]:
" ".join(x.split())

'thanks for watching and please like this video'

In [97]:
df['twitts'] = df['twitts'].apply(lambda x: " ".join(x.split()))

In [98]:
df.sample(5)

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts,emails,emails_count,urls_flag
694735,jerseyjoeygirl good luck movin in the rain ble...,0,17,93,4.411765,5,0,1,0,0,[],0,0
340378,tempted to buy been meaning to buy mintyboost ...,0,25,138,4.520000,8,0,0,0,0,[],0,1
306537,followed by 72,0,3,15,4.000000,1,0,0,1,0,[],0,0
1486139,paulblg have fun poolie,4,4,26,5.500000,1,0,1,0,0,[],0,0
256283,ugh i dont feel good im just gonnawatch more m...,0,21,106,4.047619,10,0,0,0,0,[],0,0


## Remove HTML tags

In [99]:
from bs4 import BeautifulSoup

In [100]:
x = '<html><h2>Thanks for watching</h2></html>'

In [101]:
BeautifulSoup(x, 'lxml').get_text()

'Thanks for watching'

In [102]:
%time
df['twitts'] = df['twitts'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

# this takes around 10 mins. 

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


## Remove Accented Chars

In [103]:
import unicodedata

In [104]:
x = 'Áccěntěd těxt'

In [105]:
def remove_accented_chars(x):
    x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return x

In [106]:
remove_accented_chars(x)

'Accented text'

In [107]:
df['twitts'] = df['twitts'].apply(lambda x: remove_accented_chars(x))

## Remove Stop Words

In [108]:
x = 'this is stop words removal code is a the an how what'

In [109]:
" ".join([t for t in x.split() if t not in STOP_WORDS])

'stop words removal code'

In [110]:
df['twitts'] = df['twitts'].apply(lambda x: " ".join([t for t in x.split() if t not in STOP_WORDS]))

In [111]:
df.sample(5)

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts,emails,emails_count,urls_flag
1296980,video photo shoot video wont shaky,4,13,88,5.615385,4,0,0,0,0,[],0,1
790089,laraplathe totally bummed going miss,0,8,49,5.125000,1,0,1,0,0,[],0,0
571887,huam ga bisa tdr ada yg ngajak ribut lg tired ...,0,17,83,3.882353,3,0,0,0,0,[],0,0
204792,everydayjourney lost help find good home,0,11,61,4.545455,3,0,1,0,0,[],0,0
1436785,tomtpc thanks t like taking pics,4,15,68,3.533333,7,0,1,0,0,[],0,0


## Convert into base or root form of word

In [112]:
nlp = spacy.load('en_core_web_sm')

In [113]:
x = 'kenichan dived times ball managed save 50 rest'

In [114]:
def make_to_base(x):
    x_list = []
    doc = nlp(x)
    
    for token in doc:
        lemma = str(token.lemma_)
        if lemma == '-PRON-' or lemma == 'be':   # see the video to understand ths line (https://www.youtube.com/watch?v=fQ_BXTghEbs&list=PLc2rvfiptPSToz3K_ozo7zrMJXqe16YUd&index=19)
            lemma = token.text
        x_list.append(lemma)
    print(" ".join(x_list))

In [115]:
make_to_base(x)

# we can see that dived became dive  managed became manage.

kenichan dive times ball manage save 50 rest


In [116]:
# it will take time to convert. between 30 mins to 1 hour. 

%time
df['twitts'] = df['twitts'].apply(lambda x: make_to_base(x))


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
switchfoot - awww bummer shoulda get david carr day d
upset update facebook texte cry result school today blah
kenichan dive times ball manage save 50 rest bound
body feel itchy like fire
nationwideclass behaving mad
kwesidei crew
need hug
loltrish hey long time yes rain bit bit lol fine thank
tatianak nope
twittera que muera
spring break plain city snow
re - pierced ear
caregive bear watch think ua loss embarrassing
octolinz16 count idk talk anymore
smarrison would have gun zac snyder doucheclown
iamjazzyfizzle wish get watch miss iamlilnicki premiere
hollis death scene hu severely watch film wry director cut
file taxis
lettya ahh i have want rent love soundtrack
fakerpattypattz oh dear drink forget table drink
alydesign day
friend call ask meet mid valley todaybut time sigh
angrybarista baked cake ate
week go hope
blagh class 8 tomorrow
hate wake people
go cry sleep watch marley
i m sad misslilly
ooooh lol leslie ok will

oh red sox game rain look forward open day
find key
charlietm know right dunno go twitter
get sore throat
labelsnotlove home town mammy call depressd pls explain y parent let 8yr old child walk hello 2009
ircmaxell - think need find well anti - depressant think paxilwellbutrin combo lose efficacy
myconnecticut restaurant call woodntap competitive eat tourney round tourney time place 2nd
bathroom wake lakin
caitlinoconnor want tacos margarhitas telll gay hellolt3
i m lonely company 22 female california
bad day betfair office
miss wait celebrate tar heel win weekend
cold want sleep
kristencampisi 4 u officially 4 round
monkey find twin will not write heabroken
missom aww know feel like yesterday work
sarahreedsc treaty define
missed brent praise band fun lead guitarist ltpoutgt
jpfurry poor john happen play fruit microwave seriously see doctor xxx
miss bff watch home away remind lt3 shout cous
mandayyy
new video card doa
feel lose naked confused jk ofno iphone
damn late fill appraisal fo

hummiemd know surprised recommend youtube tomorrow figure
wait airpo ride harass 2 man try sell ugly hat want sleep
leave ids location bar fuckkkkkkkkkkkkk
celycarmo sorry understand comment
officialrandl announcement stay late night
kishorek strange illegal torrent avlbl legal dvd stock
imiss
trolley packed like sardine padre game remove car good mts
randomlynat jeez try help haha
supamagg happen saturday night glittery green light
try reduce number cigarette day
yea quiet cuz work i m bored death talk
get to cook assignment hard
treesahquiche okay applepear talk people tell
jennifermf know night owl nature hahaha time zone 1
mamasvan lol - nope complete camerafail
maxime68 megeld useless sleep fair want fun
pookie uti nurse gabbie 100 soda
jillianfish tweet damn hang
anistorm
video games matter yesterdays news
sleeptilnoon leave get tempt eat soon get home chocolate velvet amp eat d mocha
stress blackberry keep flash red ale status reason faking hate
- wanna wear doc maen wear decemb

moony394 ohh omg lmao cry right lol kutnerrrr well
come running take showerwhy low stomach hu exercise
watch fallon mom work projectschool hard
56 min till home time count craving visit mac store close time
stephenkruiser sorry hear care
paulcole hmmmgreed good motivate individual well - not take away
gabbyisactive brat rub want mocha
kutnerrrr think 13 ugh kutnerkal penn - you have bright star ho
redvinylgirl mom wish good luck
hate homework -- head hus bad
get ebay blue item want jump bidder 100 hour 3 hour well hope
gigdiary know - little depressed ate night leftover today
pc living room suffer fatal paion lose itune library go doc pic
sad day manu season
stephenkruiser poor spunkyit sadmy rotti 10 pathi know feel
i m sooo sad right need hug
want step brother home i m worried do not know hope know care
heidimontag get watch hill love excited episode i m sad season
ryanmwilson aww stinks hugs
tired feel like crap world feel crummy happy usb disco mouse
do not want believe i m reading

4 year old 18 year old drive twisti want like
suck
place 3 daysany help packingpaintingcleane appreciate oh rip wall
open paially frozen beer beer toaster oven counter kitchen smell like run speakeasy
stewiebrittany do not know ride
monasmith sadly yes think need councile
make ready schooldo feel like go test
smugness go train failed turn head victoria follow unpleasant tube ride city
anybody know ipod go
aww eahquake italy sad see pics news terrible
lilibat able afford extraction continue deal
pet care people try run business send suspicious e - mail fishing info
kameelahwrites lt33 codezthey mia tho
reannaremick do not work cell sleep p
go nice tomorrow rain come nice weather bihday
taliasunset rock band hardon expe
beachbassbone roomie home day scratch door comfoer goto large laundromat machine
hard thing schedule chat end day usually
kill character favorite show upset
jealous friend get new star trek film austin tonight
listen q get bad headache drivin lesson min lucky wana sleeep


...

boring game fun
sleep worky work tomorrow 10 - 5
sad happen madonna paps cause fall insensitive
angel miss brother
physics
djsoulnyc fave aussie lol - go come mansion tonite end ordering room service time
qotd quotit take million effos build relationship fatal error ruin allquot true sad
long day sleep
pretty tired wake tomorrow stupid paper serotonin
mainaronsen nah excerciceeeeeee
viksaxena feel youboston rain
april foollsss hehe aprill follyyy timee
malareignz hahaha blush easy listen music flick roro get red cheek
kriwho bed lady know i m missin
jessicasimpson - poor thing fish die holiday
wanna country
sexual frustrated seattle
sundae delicious thought b ah welli add 2 list lose soldier pregnancy battle
hate fiberglass
3 wide awake
10 billion shopping cas
take long nap amp waste beautiful day
cupcakeali hahahaha effin awesome dude talk girl friend meet chris say rude
get home long night i m tired
gahhh study hard possibly write dot point summary
ohh sleepy95 awake lot
shanedawson 

longgggg day finally get bed work
stupid banking cheque credit soon maaaaaan
hea aches jealousy wad make bad miss tink love
hour week holiday work tomorrow
racheleh want warm bed travel
talk benjamin miss
twitterberry respond mobile net fine meeting city amf ready life soon miss la
cathbx working
teelajbrown weather la spend hour changeover plain kauai like lax
miasmatic7 look lasst post bjp
btwto trancer good pay come upi sad bas ampram break pay especially asot400
ddlovato wish conce
sit schipol airpo waiting hometo work
dahowlett maybe vet today depend long poop good
nicolerichie funnybut happy ending
damn beautiful orchid good friend go flower esta
ihenpecked tonight figure blazer game team cheer
stress bloody assignment hate woot finished tourism 2 tafe assignment goooo daaaammm
sushi awesome usual expensive remember nice
kweenie yep need permanent people whinge free vector blog
grey gardens make sad worried end like parent house hill
gatwick airpo fly friend parking attendant tho

get ready 4 skool tomorrow
kitty parking garage get home hope run
nose ring fall worked day finally homewent jacuzzi time rest finally
i m sick 3 a.m. saturday night i m homework suck
sinnamonlove - damn baby girl time favorite lookin bust pic cig
fix account picture twitter pro april fool thing get lot people track date realize
ok that s 2 night row whataburger fked
race staedleader pacecar f1
go officework today drool pentax k200
omg finally new push daisy 2 episode leave good tv series evarrrrrr
yup officiali go to hungover yankees game tmrw btwfireball whiskey taste like candyit go to 1 hot summer
watch twilight vanessa tired edward lol foot ton ant bite sleepygoodnight
kavp happy sick
go try go sleep darn caffeine keeping
grand prix safety car sta
nicthaquik feel boo neeeeeed studio like asap mann
long
greggrunberg love story lack

nicolerichie soooo good cry
baby sick make sad
good stay curious find bitter truth
parra eel suck lately grrrrrr recon group 10 yr old girl well right


cut bit skin finger owwwiiieeee think time bed
grocery massive headachenot fun
suck
apament suspicious ppl case car get photo amp plate tho half expect 2 vandalize
lucasricharz you re stay watch tv study
vautrin wow - slap person homophobe suck
wendywing staed raining mt eden
mynameislaurrra rain rain rain - staed safety car f1
jeffreburn thanx barely 140 take long login password set
pcam miss pussycatisland
dasexybkma
hot singapore 2 day
great tonight stoke silva win obvs sick bar
mileycyrus hey hun could not imagine travel like head inspiration live
smell dinner eaten sit afternoon
want chipotleand meds
basicallyim love xand i m talk newhe nice do not know wat
miss friend
go tonight cause rain
wernshen boring sta race
sooo hungry ready stuff
arcanine fail name end
sleep work weekend graveyard shift
get early watch f1 rain sc like watch traffic m25 wet monday morning

driven morecambe sun shine sunderland
sum1sstar like bitch lot i m sorry
think domain use
boo leave half night fast
bt

KeyboardInterrupt: 

In [117]:
df.sample(5)

,twitts,sentiment,words_counts,char_counts,avg_word_len,stop_words_len,hashtags_count,mentions_count,numerics_count,upper_counts,emails,emails_count,urls_flag
1514941,following thebuddha thanks tachacres - great l...,4,14,86,5.142857,4,0,2,0,0,[],0,0
327021,better docleaning,0,4,33,7.250000,2,0,0,0,0,[],0,0
652234,jst played guitar 12 hour nd finger bleed dnt ...,0,28,138,3.928571,11,0,0,0,0,[],0,0
484138,welcome monday glad tina great weekend,0,9,51,4.555556,3,0,0,0,0,[],0,0
1025828,revrachros thought beoh miss place tho credit ...,4,15,86,4.733333,3,0,1,0,0,[],0,0


## Common words removal

In [118]:
df.head()['twitts']

0    switchfoot - awww bummer shoulda got david car...
1    upset update facebook texting cry result schoo...
2    kenichan dived times ball managed save 50 rest...
3                           body feels itchy like fire
4                         nationwideclass behaving mad
Name: twitts, dtype: object

In [119]:
# Join few twitts for example 

' '.join(df.head()['twitts'])

'switchfoot - awww bummer shoulda got david carr day d upset update facebook texting cry result school today blah kenichan dived times ball managed save 50 rest bounds body feels itchy like fire nationwideclass behaving mad'

In [120]:
# Join all the twitts

text = ' '.join(df['twitts'])

In [121]:
text = text.split()

In [122]:
text

# It has got split has individual words 

['switchfoot',
 '-',
 'awww',
 'bummer',
 'shoulda',
 'got',
 'david',
 'carr',
 'day',
 'd',
 'upset',
 'update',
 'facebook',
 'texting',
 'cry',
 'result',
 'school',
 'today',
 'blah',
 'kenichan',
 'dived',
 'times',
 'ball',
 'managed',
 'save',
 '50',
 'rest',
 'bounds',
 'body',
 'feels',
 'itchy',
 'like',
 'fire',
 'nationwideclass',
 'behaving',
 'mad',
 'kwesidei',
 'crew',
 'need',
 'hug',
 'loltrish',
 'hey',
 'long',
 'time',
 'yes',
 'rains',
 'bit',
 'bit',
 'lol',
 'fine',
 'thanks',
 'tatianak',
 'nope',
 'twittera',
 'que',
 'muera',
 'spring',
 'break',
 'plain',
 'city',
 'snowing',
 're-pierced',
 'ears',
 'caregiving',
 'bear',
 'watch',
 'thought',
 'ua',
 'loss',
 'embarrassing',
 'octolinz16',
 'counts',
 'idk',
 'talk',
 'anymore',
 'smarrison',
 'wouldve',
 'gun',
 'zac',
 'snyders',
 'doucheclown',
 'iamjazzyfizzle',
 'wish',
 'got',
 'watch',
 'miss',
 'iamlilnicki',
 'premiere',
 'hollis',
 'death',
 'scene',
 'hu',
 'severely',
 'watch',
 'film',
 'wry'

In [123]:
freq_comm = pd.Series(text).value_counts()

In [124]:
f20 = freq_comm[:20]

In [125]:
f20

# These are the top 20 common words

good      89366
day       82299
like      77735
-         69663
today     64512
going     64078
love      63421
work      62804
got       60749
time      56081
lol       55094
know      51172
im        50147
want      42070
new       41995
think     41040
night     41029
amp       40616
thanks    39311
home      39168
dtype: int64

In [126]:
df['twitts'] = df['twitts'].apply(lambda x: " ".join([t for t in x.split() if t not in f20]))

## Rare words removal

In [127]:
rare20 = freq_comm[-20:]

In [128]:
rare20

glutensbye                 1
wwwmyspacecomzoesuicide    1
kboxing                    1
piratenik                  1
misskeke12                 1
costructionyou             1
likeyea                    1
julyno                     1
propaaa                    1
likefun                    1
leilas                     1
in-joy                     1
lbeezysod                  1
songsim                    1
idnw                       1
joeniar                    1
mrcuervo                   1
spake                      1
hulike                     1
lmffao                     1
dtype: int64

In [129]:
rare = freq_comm[freq_comm.values == 1]

In [130]:
rare

joyologist        1
siouxsie07        1
quotboomquot      1
accruals          1
uploadinggotta    1
                 ..
joeniar           1
mrcuervo          1
spake             1
hulike            1
lmffao            1
Length: 535435, dtype: int64

In [131]:
df['twitts'] = df['twitts'].apply(lambda x: ' '.join([t for t in x.split() if t not in rare20]))

## Word Cloud Visualization

In [132]:
# !pip install wordcloud

# Not able to install this package. Not so important feature also, so skipping. 

# Can see the video once for understanding.

# https://www.youtube.com/watch?v=oQRCxYePSJ0&list=PLc2rvfiptPSToz3K_ozo7zrMJXqe16YUd&index=22

  Using cached wordcloud-1.8.1.tar.gz (220 kB)
  ERROR: Command errored out with exit status 1:
   command: /Users/yogesh/opt/anaconda3/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/1t/9cvzqdfs2m7d80tj_25jwk9m0000gn/T/pip-install-2a1wggm2/wordcloud_950cba09f8fb454394c0ef38e61e348f/setup.py'"'"'; __file__='"'"'/private/var/folders/1t/9cvzqdfs2m7d80tj_25jwk9m0000gn/T/pip-install-2a1wggm2/wordcloud_950cba09f8fb454394c0ef38e61e348f/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/1t/9cvzqdfs2m7d80tj_25jwk9m0000gn/T/pip-wheel-fta4mnq3
       cwd: /private/var/folders/1t/9cvzqdfs2m7d80tj_25jwk9m0000gn/T/pip-install-2a1wggm2/wordcloud_950cba09f8fb454394c0ef38e61e348f/
  Complete output (24 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/l

In [249]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'wordcloud'

In [ ]:
x = ' '.join(text[:20000])

In [ ]:
wc = WordCloud(width = 800, height=400).generate(x)
plt.imshow(wc)
plt.axis('off')
plt.show()

## Spelling Correction

In [252]:
# pip install -U textblob

# Need to install via terminal. Not here. 

SyntaxError: invalid syntax (<ipython-input-252-5cfc515c9dee>, line 2)

In [253]:
# python -m textblob.download_corpora

# Need to install via terminal. Not here. 

SyntaxError: invalid syntax (<ipython-input-253-bec4ba3f7ac1>, line 1)

In [133]:
from textblob import TextBlob

In [134]:
x = 'tanks forr waching this vidio carri'

In [135]:
x = TextBlob(x).correct()

In [136]:
x

TextBlob("tanks for watching this video carry")

In [190]:
# we can apply the same to whole twitts, as it will take time have not applied here. 

# df['twitts'] = df['twitts'].apply(lambda x: TextBlob(x).correct())

### Export the dataset for future use

In [199]:
df.to_csv('1_DSH_Basic_Text_Mining_Output_v1.csv')

In [206]:
# df.to_csv(r"Documents\Bharadwaj's\Career\Data Science\Datasets\1_DSH_Basic_Text_Mining_Output_v1.csv")

# File is not getting created in the path provided, not sure why. It creates in the current folder only. 
# Need to check later. 
# have manually moved the file to dataset folder. 

# Need to try once by giving path as below. 

# "http:\\localhost:8888\tree\Documents\Bharadwaj's\Career\Data Science\Handsets\1_DSH_Basic_Text_Mining_Output_v1.csv"

## Tokenization - breaking the sentences into individual words.

In [138]:
x = 'thanks#watching this video. please like it'

In [139]:
TextBlob(x).words

WordList(['thanks', 'watching', 'this', 'video', 'please', 'like', 'it'])

In [140]:
doc = nlp(x)

In [141]:
doc

thanks#watching this video. please like it

In [142]:
for token in doc:
    print(token)

thanks#watching
this
video
.
please
like
it


## Lemmatization

In [143]:
'''
Lemmatization is similar to stemming but it brings context to the words. 
So it links words with similar meaning to one word.
'''

'\nLemmatization is similar to stemming but it brings context to the words. \nSo it links words with similar meaning to one word.\n'

In [144]:
x = 'runs run running ran'

In [145]:
from textblob import Word

In [146]:
for token in x.split():
    print(Word(token).lemmatize())

run
run
running
ran


In [147]:
doc = nlp(x)
for token in doc:
    print(token.lemma_)

run
run
run
run


## Detect Entities using NER of SpaCy


Named Entity Recognition (NER) is a standard NLP problem which involves spotting named entities (people, places, 
organizations etc.) from a chunk of text, and classifying them into a predefined set of categories. 

Some of the practical applications of NER include:

Scanning news articles for the people, organizations and locations reported.
Providing concise features for search optimization: instead of searching the entire content, one may simply search 
for the major entities involved.
Quickly retrieving geographical locations talked about in Twitter posts.


In [150]:
x = "Breaking News: Donald Trump, the president of the USA is looking to sign a deal to mine the moon"

In [151]:
doc = nlp(x)

for ent in doc.ents:
    print(ent.text + ' - ' + ent.label_ + ' - ' + str(spacy.explain(ent.label_)))

Donald Trump - PERSON - People, including fictional
USA - GPE - Countries, cities, states


In [152]:
from spacy import displacy

In [153]:
displacy.render(doc, style = 'ent')

## Detecting Nouns

In [154]:
x

'Breaking News: Donald Trump, the president of the USA is looking to sign a deal to mine the moon'

In [155]:
for noun in doc.noun_chunks:
    print(noun)

Breaking
News
Donald Trump
the president
the USA
a deal
the moon


## Translation and Language Detection

In [156]:
x

'Breaking News: Donald Trump, the president of the USA is looking to sign a deal to mine the moon'

In [157]:
tb = TextBlob(x)

In [158]:
tb

TextBlob("Breaking News: Donald Trump, the president of the USA is looking to sign a deal to mine the moon")

In [159]:
tb.detect_language()

'en'

In [160]:
tb.translate(to='kn')

# Language Code: https://www.loc.gov/standards/iso639-2/php/code_list.php

TextBlob("ಬ್ರೇಕಿಂಗ್ ನ್ಯೂಸ್: ಯುಎಸ್ಎ ಅಧ್ಯಕ್ಷ ಡೊನಾಲ್ಡ್ ಟ್ರಂಪ್ ಚಂದ್ರನನ್ನು ಗಣಿಗಾರಿಕೆ ಮಾಡಲು ಒಪ್ಪಂದಕ್ಕೆ ಸಹಿ ಹಾಕಲು ನೋಡುತ್ತಿದ್ದಾರೆ")

# Use inbuilt sentiment classifier

In [161]:
# TextBlob library also comes with a NaiveBayesAnalyzer, Naive Bayes is a commonly used 
# machine learning text-classification algorithm.

In [162]:
from textblob.sentiments import NaiveBayesAnalyzer

In [163]:
x = 'we all stands together to fight with corona virus. we will win together'

In [164]:
tb = TextBlob(x, analyzer=NaiveBayesAnalyzer())

In [165]:
tb.sentiment

Sentiment(classification='pos', p_pos=0.8259779151942094, p_neg=0.17402208480578962)

In [166]:
x = 'we all are sufering from corona'

In [167]:
tb = TextBlob(x, analyzer=NaiveBayesAnalyzer())

In [168]:
tb.sentiment

# Supposed to be negative but classification stil came as positive. 

Sentiment(classification='pos', p_pos=0.75616044472398, p_neg=0.2438395552760203)

In [169]:
x = 'Many deaths happening due to corona'

In [170]:
tb = TextBlob(x, analyzer=NaiveBayesAnalyzer())

In [171]:
tb.sentiment

# Supposed to be negative but classification stil came as positive. 

Sentiment(classification='pos', p_pos=0.8363425287463141, p_neg=0.16365747125368613)

In [172]:
x = 'corona is very worst'

In [173]:
tb = TextBlob(x, analyzer=NaiveBayesAnalyzer())

In [174]:
tb.sentiment

# Supposed to be negative , classification also came as negative. 

Sentiment(classification='neg', p_pos=0.4564051188205746, p_neg=0.5435948811794252)

In [175]:
x = 'corona is very very very worst'

In [176]:
tb = TextBlob(x, analyzer=NaiveBayesAnalyzer())

In [177]:
tb.sentiment

# since the word very doesnt reflect negativity, so repeating very doesnt increases negative score. 

Sentiment(classification='neg', p_pos=0.4564051188205746, p_neg=0.5435948811794252)

In [183]:
x = 'corona is very worst, worst, worst'

In [184]:
tb = TextBlob(x, analyzer=NaiveBayesAnalyzer())

In [185]:
tb.sentiment

#now also the score remains same becasuse the word worst is already considered for negative score. But it would have
# increased some more -ve score but didnt happen. 

Sentiment(classification='neg', p_pos=0.4564051188205746, p_neg=0.5435948811794252)

In [186]:
x = 'corona is very worst, very bad, most killing'

In [187]:
tb = TextBlob(x, analyzer=NaiveBayesAnalyzer())

In [188]:
tb.sentiment

# as we have 3 words with negative meaning, the -ve score has increased. 

Sentiment(classification='neg', p_pos=0.25737188376777126, p_neg=0.7426281162322292)